In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [4]:
# On English wikipedia (May 2020)

wiki = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/','enwiki_20200520_custom.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [7]:
## Not currently getting a paragraph = an article, we have additional blank lines coming through
i = 0
for p in wiki.paras():
    if i > 5: break
    print(p)
    i +=1 

[['Anarchism', 'is', 'a', 'political', 'philosophy', 'and', 'movement', 'that', 'rejects', 'all', 'involuntary,', 'coercive', 'forms', 'of', 'hierarchy.'], ['It', 'radically', 'calls', 'for', 'the', 'abolition', 'of', 'the', 'state', 'which', 'it', 'holds', 'to', 'be', 'undesirable,', 'unnecessary', 'and', 'harmful.'], ['The', 'timeline', 'of', 'anarchism', 'stretches', 'back', 'to', 'prehistory,', 'when', 'people', 'lived', 'in', 'anarchistic', 'societies', 'long', 'before', 'the', 'establishment', 'of', 'formal', 'states,', 'kingdoms', 'or', 'empires.'], ['With', 'the', 'rise', 'of', 'organised', 'hierarchical', 'bodies,', 'skepticism', 'towards', 'authority', 'also', 'rose,', 'but', 'it', 'was', 'not', 'until', 'the', '19th', 'century', 'that', 'a', 'self-conscious', 'political', 'movement', 'was', 'formed.'], ['During', 'the', 'later', 'half', 'of', 'the', '19th', 'and', 'the', 'first', 'decades', 'of', 'the', '20th', 'century,', 'the', 'anarchist', 'movement', 'flourished', 'in', 

In [8]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [9]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [10]:
scorer = NgramAssocMeasures.poisson_stirling

bi_dict = {}
tri_dict = {}

In [11]:
%%time
bi_cf = BigramCollocationFinder.from_words(word_gen(wiki))
bi_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [ ]:
bi_cf.N

In [ ]:
import pickle

In [ ]:
with open(datapath+'/Corpora/wiki/enwiki_20200520/Pickles/bi_cf.pkl','wb') as pickfile:
    pickle.dump(bi_cf,pickfile)

In [ ]:
#Less memory-intensive?
for bigram in bi_cf.ngram_fd.items():
     bi_dict[bigram[0]] = [bigram[0], bigram[-1], bi_cf.score_ngram(scorer,bigram[0][0],bigram[0][1])]

In [12]:
#for bigram in bi_cf.score_ngrams(scorer):
#    bi_dict[bigram[0]] = [bigram[0], bi_cf.ngram_fd[bigram[0]], bigram[-1]]

MemoryError: 

In [ ]:
del bi_cf

In [ ]:
bigram_df = pd.DataFrame.from_dict(bi_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

In [ ]:
# Remove dictionaries
del bi_dict

In [ ]:
%%time
tri_cf = TrigramCollocationFinder.from_words(word_gen(wiki))
tri_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [ ]:
for trigram in tri_cf.score_ngrams(scorer):    
    tri_dict[trigram[0]] = [trigram[0], tri_cf.ngram_fd[trigram[0]], trigram[-1]]

In [ ]:
del tri_cf

In [ ]:
trigram_df = pd.DataFrame.from_dict(tri_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

In [ ]:
# Remove dictionaries
del tri_dict

In [ ]:
ngram_df = bigram_df.append(trigram_df).sort_values('poisson', ascending=False).reset_index(drop=True)

In [ ]:
# Remove separate instances
del bi_df
del tri_df

In [ ]:
ngram_df['len'] = ngram_df.ngram.apply(len)

In [ ]:
ngram_df

In [ ]:
#len(wiki.words())

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [ ]:
%%time

# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(wiki, sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [ ]:
stop

In [ ]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [ ]:
min_freq = 20

In [ ]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

In [ ]:
# Clean up
del ngram_df

In [ ]:
eval_count = 1000000

ngram_eval = ngram_df2[0:eval_count]

ngram_eval

In [ ]:
# Clean up
del ngram_df2

In [ ]:
%%time

batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 10)

batches

In [ ]:
len(batches)

In [ ]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

In [ ]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

In [ ]:
# Cleanup
del rows, ng_batch

In [ ]:
ngram_eval.batch.value_counts()

In [ ]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [ ]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    sents_mwe = Sent_Seq(wiki, batch_token_mwe)
    
    batch_model = Word2Vec(sents_mwe,
                             min_count = 20,  # 20 matches R&E on EN Wiki
                             size = 400,
                             workers = 8,
                             window = 5,
                             sg = 0,         # CBOW
                             sample = 10e-5, # Subsampling
                             negative = 10
                            )

    # Save model
    batch_model.save(datapath+'/Models/1 w2v/wiki20_batch{}.model'.format(bb+1))
    # Reload looks like    new_model = gensim.models.Word2Vec.load('/tmp/mymodel')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim', 'base_nearest', 'mwe_nearest'])

    for exp in batch_dfs[bb].ngram:
        # Combined token for MWE
        mwetoken = '+'.join(exp)
        
        # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
        sws = []
        # Component vectors
        cvs = []

        #  Neighbours in original & MWE-aware space
        oldn = []
        newn = []
      
        # List of individual word similarities (where present in the vocab)
        css = []
        
        # Empty array
        earr = np.empty(1000)
        earr[:] = np.nan
        
        # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
        #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
        if mwetoken in batch_model.wv.vocab:
            
            mwv = batch_model.wv[mwetoken]

            for w in exp:
                if w in batch_model.wv.vocab:
                    cvs.append(batch_model.wv[w])

                    oldn.append(batch_model.wv.most_similar(w, topn=5))

                    if w in stop:
                        sws.append(1)
                        css.append(np.nan)
                    else:
                        sws.append(0)
                        css.append(cosim(batch_model.wv[w], mwv ))

                # If component is absent from vocab
                else:
                    sws.append(-1)
                    cvs.append(earr)
                    css.append(np.nan)

                    oldn.append([])


            #  Mean cosim
            if min(sws) >= 0:
                cs = np.nanmean(css)
            else:
                cs = np.nan

            newn = batch_model.wv.most_similar(mwetoken, topn=5)
            
        # Combined token missing from vocab - mark with defaults
        else:
            sws = [-2]
            mwv = np.empty(400)
            mwv[:] = np.nan

            
        # Append to stats df
        statsf = statsf.append({
            'ngram'  : exp,
            'stopwords' : sws,
            'mwe_vector' : mwv,
            'component_vectors' : cvs,
            'component_cosims'  : css,
            'cosine_sim'  : cs,
            'base_nearest': oldn,
            'mwe_nearest' : newn,
        }, ignore_index=True)

    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    
    
batch_dfs

In [ ]:
# Model inspection
batch_model.wv.similarity('April', 'August')
batch_model.wv.similarity('woman', 'man')
batch_model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
batch_model.wv.most_similar('relationships', topn=5)

In [ ]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Results/wiki20_output_001.csv', index=False)

In [ ]:
all_batches